In [79]:
import pandas as pd
import numpy as np

In [80]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')
df.describe()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,num_doors,fuel_efficiency_mpg
count,9704.000000,9222.000000,8996.000000,9704.000000,8774.000000,9704.000000,9202.000000,9704.000000
mean,199.708368,3.962481,149.657292,3001.280993,15.021928,2011.484027,-0.006412,14.985243
std,49.455319,1.999323,29.879555,497.894860,2.510339,6.659808,1.048162,2.556468
min,10.000000,0.000000,37.000000,952.681761,6.000000,2000.000000,-4.000000,6.200971
25%,170.000000,3.000000,130.000000,2666.248985,13.300000,2006.000000,-1.000000,13.267459
50%,200.000000,4.000000,149.000000,2993.226296,15.000000,2012.000000,0.000000,15.006037
75%,230.000000,5.000000,170.000000,3334.957039,16.700000,2017.000000,1.000000,16.707965
max,380.000000,13.000000,271.000000,4739.077089,24.300000,2023.000000,4.000000,25.967222


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   num_cylinders        9222 non-null   float64
 2   horsepower           8996 non-null   float64
 3   vehicle_weight       9704 non-null   float64
 4   acceleration         8774 non-null   float64
 5   model_year           9704 non-null   int64  
 6   origin               9704 non-null   object 
 7   fuel_type            9704 non-null   object 
 8   drivetrain           9704 non-null   object 
 9   num_doors            9202 non-null   float64
 10  fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(6), int64(2), object(3)
memory usage: 834.1+ KB


In [82]:
df = df[[
    'engine_displacement',
    'horsepower',
    'vehicle_weight',
    'model_year',
    'fuel_efficiency_mpg'
        ]]

#Question 1
#There's one column with missing values. What is it?
df.info()
#horsepower

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9704 entries, 0 to 9703
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine_displacement  9704 non-null   int64  
 1   horsepower           8996 non-null   float64
 2   vehicle_weight       9704 non-null   float64
 3   model_year           9704 non-null   int64  
 4   fuel_efficiency_mpg  9704 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 379.2 KB


In [83]:
#Question 2
#What's the median (50% percentile) for variable 'horsepower'?
df.horsepower.median()

np.float64(149.0)

In [84]:
#Prepare and split the dataset
#    Shuffle the dataset (the filtered one you created above), use seed 42.
#    Split your data in train/val/test sets, with 60%/20%/20% distribution.
#Use the same code as in the lectures

np.random.seed(42)
n = len(df)
n_val = n_test = int(n*.2)
n_train = n - (n_val + n_test)
np.random.shuffle(idx := np.arange(n))

df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]



In [85]:
print((len(df_train), len(df_val), len(df_test)))
print(sum((len(df_train), len(df_val), len(df_test))))
print(len(df))

(5824, 1940, 1940)
9704
9704


In [86]:
y_train = df_train.fuel_efficiency_mpg.values; del df_train['fuel_efficiency_mpg']
y_val = df_val.fuel_efficiency_mpg.values; del df_val['fuel_efficiency_mpg']
y_test = df_test.fuel_efficiency_mpg.values; del df_test['fuel_efficiency_mpg']

In [87]:
y_train

array([16.64294342, 16.29837715, 18.59182197, ..., 17.59323665,
       18.92574819, 17.96528447], shape=(5824,))

In [88]:
'''Question 3

    We need to deal with missing values for the column from Q1.
    We have two options: fill it with 0 or with the mean of this variable.
'''

ave_hp = df_train.horsepower.mean()

df_train0 = df_train.fillna(0, inplace=False)
df_trainA = df_train.fillna(ave_hp, inplace=False)
df_test0 = df_test.fillna(0, inplace=False)
df_testA = df_test.fillna(ave_hp, inplace=False)
df_val0 = df_val.fillna(0, inplace=False)
df_valA = df_val.fillna(ave_hp, inplace=False)

X0 = df_train0.values
XA = df_trainA.values


#df_train0.info()
X0

array([[ 220.        ,  144.        , 2535.88759124, 2009.        ],
       [ 160.        ,  141.        , 2741.17048439, 2019.        ],
       [ 230.        ,  155.        , 2471.88023726, 2017.        ],
       ...,
       [ 210.        ,  152.        , 2500.17568746, 2020.        ],
       [ 250.        ,  154.        , 2254.31324473, 2002.        ],
       [ 210.        ,  152.        , 2435.20360628, 2004.        ]],
      shape=(5824, 4))

In [89]:
'''
    Try both options. For each, train a linear regression model without regularization using the code from the lessons.
    For computing the mean, use the training only!
'''

def train_linear_regression(X, y):
    X = np.column_stack([np.ones(X.shape[0]), X])

    #XTX = X.T @ X
    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    #w_full = (XTX_inv @ X.T) @ y
    w_full = XTX_inv.dot(X.T).dot(y)
    return w_full 
 
def predict(X, w):
    X = np.column_stack([np.ones(X.shape[0]), X])
    return X.dot(w)

def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [90]:
'''
Use the validation dataset to evaluate the models and compare the RMSE of each option.
    Round the RMSE scores to 2 decimal digits using round(score, 2)
    Which option gives better RMSE?
'''

w0 = train_linear_regression(X0, y_train)
wA = train_linear_regression(XA, y_train)
print(w0)

y_pred0 = predict(df_val.values, w0)
y_predA = predict(df_val.values, wA)
print(np.isnan(y_pred0).sum(), len(y_pred0))
y_pred0 = np.nan_to_num(y_pred0, nan=0)#np.nanmean(y_pred0))
y_predA = np.nan_to_num(y_predA, nan=0)#np.nanmean(y_predA))


rmse0 = rmse(y_val, y_pred0)
rmseA = rmse(y_val, y_predA)

print(round(rmse0, 2), round(rmseA, 2))

[ 2.88273655e+01  9.93613218e-05  3.54221891e-03 -5.01328015e-03
  3.42992870e-04]
136 1940
4.03 4.03


In [91]:
'''
Question 4

    Now let's train a regularized linear regression.
    For this question, fill the NAs with 0.
    Try different values of r from this list: [0, 0.01, 0.1, 1, 5, 10, 100].
    Use RMSE to evaluate the model on the validation dataset.
    Round the RMSE scores to 2 decimal digits.
    Which r gives the best RMSE?

If multiple options give the same best RMSE, select the smallest r.
'''

def train_linear_regression_reg(X, y, r=0.001):
    X = np.column_stack([np.ones(X.shape[0]), X])
    
    XTX = X.T.dot( X)
    XTX = XTX + r * np.eye(XTX.shape[0])
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot( X.T).dot(y)
    return w_full 

for r in [0, 0.01, 0.1, 1, 5, 10, 100]:
    w0 = train_linear_regression_reg(X0, y_train, r)
    
    y_pred0 = predict(df_val.values, w0)
 #   print(np.isnan(y_pred0).sum(), len(y_pred0))
    y_pred0 = np.nan_to_num(y_pred0, nan=0)#np.nanmean(y_pred0))
        
    print('r = ', r, ' --> ', round(rmse(y_val, y_pred0), 2))
   





r =  0  -->  4.03
r =  0.01  -->  4.03
r =  0.1  -->  4.03
r =  1  -->  4.03
r =  5  -->  4.03
r =  10  -->  4.03
r =  100  -->  4.03


In [92]:
'''
Question 5

    We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
    Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
    For each seed, do the train/validation/test split with 60%/20%/20% distribution.
    Fill the missing values with 0 and train a model without regularization.
    For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
    What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
    Round the result to 3 decimal digits (round(std, 3))

df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv')

df = df[[
    'engine_displacement',
    'horsepower',
    'vehicle_weight',
    'model_year',
    'fuel_efficiency_mpg'
        ]]



n = len(df)
n_val = n_test = int(n*.2)
n_train = n - (n_val + n_test)
'''

def sow(seed):
    np.random.seed(seed)
    np.random.shuffle(idx := np.arange(n))
    #print(idx[:6])
    df_train = df.iloc[idx[:n_train]]
    df_val = df.iloc[idx[n_train:n_train+n_val]]
    df_test = df.iloc[idx[n_train+n_val:]]

    return df_train, df_val, df_test



def cultivate(seeds, r = 0):
    rmses = []
    for seed in seeds:
        df_train, df_val, df_test = sow(seed)
        
        y_train = df_train.fuel_efficiency_mpg.values; del df_train['fuel_efficiency_mpg']
        y_val = df_val.fuel_efficiency_mpg.values; del df_val['fuel_efficiency_mpg']
        y_test = df_test.fuel_efficiency_mpg.values; del df_test['fuel_efficiency_mpg']
    
        df_train0 = df_train.fillna(0, inplace=False)
        df_test0 = df_test.fillna(0, inplace=False)
        df_val0 = df_val.fillna(0, inplace=False)
    
        X0 = df_train0.values

        w0 = train_linear_regression_reg(X0, y_train, r)


        y_pred0 = predict(df_val.values, w0)
        #print(np.isnan(y_pred0).sum(), len(y_pred0))
        y_pred0 = np.nan_to_num(y_pred0, nan=0)#np.nanmean(y_pred0))
        
        print(rmse(y_val, y_pred0))
        rmses.append(rmse(y_val, y_pred0))


    return rmses





In [93]:
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

print(np.std(cultivate(seeds)))

4.115363569087568
4.0205487806355515
4.034964418667787
4.239725265476986
4.056847484026755
4.148705656727751
4.1908952458782815
3.892524555387156
4.18874290991515
4.014237971474517
0.100062276678094


In [94]:
'''
Question 6

    Split the dataset like previously, use seed 9.
    Combine train and validation datasets.
    Fill the missing values with 0 and train a model with r=0.001.
    What's the RMSE on the test dataset?

'''
df_train, df_val, df_test = sow(9)

df_new_train = pd.concat([df_train, df_val])

df_new_train0 = df_new_train.fillna(0, inplace=False)
y_new_train = df_new_train0.fuel_efficiency_mpg.values; del df_new_train0['fuel_efficiency_mpg']
y_test = df_test.fuel_efficiency_mpg.values; del df_test['fuel_efficiency_mpg']

df_test0 = df_test.fillna(0, inplace=False)
 
X0 = df_new_train0.values

w0 = train_linear_regression_reg(X0, y_new_train, 0.001)


y_pred0 = predict(df_test0.values, w0)
#print(np.isnan(y_pred0).sum(), len(y_pred0))
y_pred0 = np.nan_to_num(y_pred0, nan=0)#np.nanmean(y_pred0))

print(rmse(y_test, y_pred0))
 

0.5156261299169602
